# ZTF Kafka Demo

## Setup:

You will need to download/acquire the authentication and configuration files: `pitt-reader.user.keytab` and `krb5.conf`


In [1]:
import os

from confluent_kafka import Consumer, KafkaException, OFFSET_BEGINNING


## Configure a consumer

- For details on configuration options see https://github.com/edenhill/librdkafka/blob/master/CONFIGURATION.md
- For API docs see https://docs.confluent.io/current/clients/confluent-kafka-python/

In [2]:
os.environ['KRB5_CONFIG'] = './krb5.conf'

# Consumer configuration
conf = {
    'bootstrap.servers': 'public2.alerts.ztf.uw.edu:9094',
    'group.id': 'group',
    'session.timeout.ms': 6000,
    'enable.auto.commit': 'FALSE',  
    'sasl.kerberos.principal': 'pitt-reader@KAFKA.SECURE',
    'sasl.kerberos.kinit.cmd': 'kinit -t "%{sasl.kerberos.keytab}" -k %{sasl.kerberos.principal}',
    'sasl.kerberos.keytab': './pitt-reader.user.keytab',
    'sasl.kerberos.service.name': 'kafka',
    'security.protocol': 'SASL_PLAINTEXT',
    'sasl.mechanisms': 'GSSAPI',
    'auto.offset.reset': 'earliest'
}

# Create Consumer instance
c = Consumer(conf, debug='fetch')


## Subscribe and retrieve an assignment

Note that each observing night is it's own topic.

In [3]:
year = 2020
month = 11
day = 19

# Each topic is its own date
topic = f'ztf_{year}{month:02d}{day:02d}_programid1'
c.subscribe([topic])


In [ ]:
# msg = c.consume(num_messages=1, timeout=5)[0] # THIS CALL JUST HANGS
# c.assignment()

c.poll(timeout=5) # THE NOTEBOOK HANGS HERE AND I END UP HAVING TO KILL THE KERNEL


In [ ]:
# Poll until our consumer is automatically assigned to a partition(s)
while not c.assignment():
    c.poll(timeout=5)

c.assignment()


In [ ]:
# If we want, we can manually reset to the beggining of each partition
topic_partitions = c.assignment()
for tp in topic_partitions:
    tp.offset = OFFSET_BEGINNING
    
c.assign(topic_partitions)


In [5]:
while not c.assignment():
    print('no')
    break

no


### Poll for data

In [ ]:
msg = None
while not msg:
    msg = c.poll(timeout=1.0)
    
if msg.error():
    raise KafkaException(msg.error())

print('topic:    ', msg.topic())
print('partition:', msg.partition())
print('offset:   ', msg.offset())
print('key:      ', str(msg.key()), '\n') 
    